In [ ]:
# %%cleanup -f

In [ ]:
# spark

In [2]:
#%stop_session

Stopping session: c3259138-394d-4aca-9d94-97985f40f5ca
Stopped session.


##### Importamos todas las funciones de pyspark que vamos a necesitar 

In [1]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, datediff, last, when, date_add, udf, lit, desc, row_number
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import OneHotEncoder, StringIndexer

BUCKET_NAME = "miriam-tmf-oct22-msf-data"

Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::342515978102:role/LabRole
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: c5f5388b-d139-496c-ad9e-916b51e3a35c
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.38.1
--enable-glue-datacatalog true
Waiting for session c5f5388b-d139-496c-ad9e-916b51e3a35c to get into ready status...
Session c5f5388b-d139-496c-ad9e-916b51e3a35c has been created.



#### Recurring Donations

##### Cargamos en un dataframe de Spark la tabla de donaciones periódicas

In [36]:
df_rec = (
  spark.read
    .option("header", "true")
    .option("quote", "\"")
    .option("escape" , "\"")   
    .format("parquet")
    .load(f"s3://{BUCKET_NAME}/MSF_RecurringDonation.parquet")
)

##### Renombramos el id como en los casos anteriores

In [37]:
df_rec = df_rec.withColumnRenamed("id","rec_id")

##### Eliminamos columnas que entendemos no deben formar parte de las features

In [38]:
# Eliminamos campos que no deben estar entre las features
columnsToDrop_rec = ["IsDeleted", "msf_CancelationReason__c", "msf_CurrentCampaign__c","npe03__Amount__c", "npe03__Last_Payment_Date__c",
                 "npe03__Next_Payment_Date__c", "npe03__Open_Ended_Status__c","npe03__Paid_Amount__c", "npe03__Total_Paid_Installments__c", "npsp4hub__Payment_Method__c"]

df_rec_f = df_rec.drop(*columnsToDrop_rec)

##### Cargamos en un dataframe de Spark la tabla de modificaciones de cuota por parte de los socios

In [23]:
df_quo = (
  spark.read
    .option("header", "true")
    .option("quote", "\"")
    .option("escape" , "\"")   
    .format("parquet")
    .load(f"s3://{BUCKET_NAME}/MSF_QuotaModification.parquet")
)

##### Renombramos el id como en los casos anteriores

In [24]:
df_quo = df_quo.withColumnRenamed("id","quo_id")

##### Eliminamos columnas que entendemos no deben formar parte de las features

In [41]:
# Eliminamos campos que no deben estar entre las features
columnsToDrop_quo = ["name", "msf_changeamount__c","msf_leadsource1__c", "msf_leadsource2__c", "msf_newamount__c", "msf_newannualizedquota__c"
                 "msf_newrecurringperiod__c"]

df_quo_f = df_quo.drop(*columnsToDrop_quo)

#### Contacts

##### Cargamos en un dataframe de Spark el fichero de Contactos de MSF

In [42]:
df_con = (
  spark.read
    .option("header", "true")
    .option("quote", "\"")
    .option("escape" , "\"")   
    .format("parquet")
    .load(f"s3://{BUCKET_NAME}/MSF_Contact.parquet")
)

##### Renombramos la columna id a con_id para que no haya ambiguedades en los merges

In [43]:
df_con = df_con.withColumnRenamed("id","con_id")

##### Nos quedamos con las columnas que nos parecen más útiles

In [44]:
df_con_f = df_con.select("msf_birthyear__c", "Gender__c", "msf_LTVCont__c", "msf_PressureComplaint__c", "msf_ScoringRFVRecurringDonor__c","msf_AverageDonorValue__c","con_id",
                        "msf_EntryCampaign__c", "msf_FirstCampaignEntryRecurringDonor__c", "msf_FirstCampaingColaboration__c")

##### Hacemos el merge entre las tablas de donaciones periódicas, la de modificaciones de cuotas, la de contactos

In [46]:
df_merge = (
    df_rec_f
    .join(df_con_f, col("npe03__Contact__c") == col("con_id"), 'inner')
    .join(df_quo_f, col("rec_id") == col("msf_recurringdonation__c"), 'inner')
    .select(
        df_rec_f["*"],
        df_con_f["*"],
        df_quo_f["*"]
    )
    .where(
        (col("npe03__Contact__c") == col("con_id")) & 
        (col("rec_id") == col("msf_recurringdonation__c")) &
        (col("con_id") == col("msf_contactid__c"))
    )
)

##### Guardamos en S3 este primer dataframe

In [47]:
(
  df_merge.write
        .format("parquet")
        .mode("overwrite")
        .save(f"s3://{BUCKET_NAME}/output/df_merge_m")
)

In [48]:
original_count = df_merge.count()
no_duplicates_count = df_merge.dropDuplicates().count()

-----------------------------------------------------------------------------------

In [49]:
print("Cantidad de filas originales:", original_count)
print("Cantidad de filas después de eliminar duplicados:", no_duplicates_count)
print("Filas duplicadas encontradas:", original_count - no_duplicates_count)

Cantidad de filas originales: 2003017
Cantidad de filas después de eliminar duplicados: 2003017
Filas duplicadas encontradas: 0


In [50]:
# Vemos si tienen valores missings:

from pyspark.sql import functions as F

columnas = df_merge.columns
# Inicializa un diccionario para almacenar los resultados
resultados_por_columna = {}
# Total de filas en el DataFrame
#original_count = df_contact.count()

# Itera sobre cada columna y aplica las condiciones
for columna in columnas:
    valores_ausentes = df_merge.filter(
        (F.col(columna).isNull()) |
        (F.col(columna) == '') |
        (F.col(columna).contains('None')) |
        (F.col(columna).contains('Null'))
    )
    
    # Cuenta el número de filas que cumplen las condiciones para esta columna
    filas_con_condiciones = valores_ausentes.count()
    
    # Almacena el resultado en el diccionario
    resultados_por_columna[columna] = filas_con_condiciones

# Muestra los resultados
for columna, conteo in resultados_por_columna.items():
    print(f"En la columna '{columna}' hay ", conteo, " filas con valores ausente, que suponen el ", round(((conteo/original_count) * 100), 2), "% del total" )

En la columna 'rec_id' hay  0  filas con valores ausente, que suponen el  0.0 % del total
En la columna 'msf_annualizedquota__c' hay  0  filas con valores ausente, que suponen el  0.0 % del total
En la columna 'msf_cancelationdate__c' hay  1037623  filas con valores ausente, que suponen el  51.8 % del total
En la columna 'msf_currentleadsource1__c' hay  5136  filas con valores ausente, que suponen el  0.26 % del total
En la columna 'msf_currentquotamodification__c' hay  2125  filas con valores ausente, que suponen el  0.11 % del total
En la columna 'msf_leadsource1__c' hay  300  filas con valores ausente, que suponen el  0.01 % del total
En la columna 'msf_memberid__c' hay  0  filas con valores ausente, que suponen el  0.0 % del total
En la columna 'npe03__contact__c' hay  4  filas con valores ausente, que suponen el  0.0 % del total
En la columna 'npe03__date_established__c' hay  0  filas con valores ausente, que suponen el  0.0 % del total
En la columna 'npe03__installment_period__c'

In [51]:
# Vemos que valores puede tomar las variables seleccionadas 

for columna in columnas:
    resultado = df_merge.groupBy(columna).count().orderBy(columna)
    print(f"Agrupación y conteo para la columna '{columna}':")
    resultado.show()

Agrupación y conteo para la columna 'rec_id':
+------------------+-----+
|            rec_id|count|
+------------------+-----+
|a093Y00001RZ7XfQAL|    5|
|a093Y00001RZ7XgQAL|    2|
|a093Y00001RZ7XhQAL|    1|
|a093Y00001RZ7XiQAL|    1|
|a093Y00001RZ7XjQAL|    7|
|a093Y00001RZ7XkQAL|    1|
|a093Y00001RZ7XlQAL|    2|
|a093Y00001RZ7XmQAL|    1|
|a093Y00001RZ7XnQAL|    6|
|a093Y00001RZ7XoQAL|    1|
|a093Y00001RZ7XpQAL|    1|
|a093Y00001RZ7XqQAL|    1|
|a093Y00001RZ7XrQAL|    1|
|a093Y00001RZ7XsQAL|    4|
|a093Y00001RZ7XtQAL|    1|
|a093Y00001RZ7XuQAL|    1|
|a093Y00001RZ7XvQAL|    3|
|a093Y00001RZ7XwQAL|    2|
|a093Y00001RZ7XxQAL|    1|
|a093Y00001RZ7XyQAL|    2|
+------------------+-----+
only showing top 20 rows

Agrupación y conteo para la columna 'msf_annualizedquota__c':
+----------------------+-----+
|msf_annualizedquota__c|count|
+----------------------+-----+
|                   0.0| 4917|
|                   0.6|    3|
|                  0.72|    9|
|                   1.0|    5|
|

### Cargamos el dataframe guardado

In [8]:
#df_merge = (
#  spark.read
#    .option("header", "true")
#    .option("quote", "\"")
#    .option("escape" , "\"")   
#    .format("parquet")
#    .load(f"s3://{BUCKET_NAME}/output/df_merge_m")
#)

In [24]:
tipos_de_columnas = df_merge.dtypes

# Hacemos dos listas para almacenar las columnas numéricas y string: 
columnas_numericas = []
columnas_string = []

for columna, tipo in tipos_de_columnas:
    if tipo == 'double':
        columnas_numericas.append(columna)
    elif tipo == 'string':
        columnas_string.append(columna)
       
if columnas_numericas:
    print("Estadísticas para columnas numéricas:")
    df_merge.select(columnas_numericas).describe().show()
else:
    print("No hay columnas numéricas en el DataFrame.")

# Hacer el summary para las columnas de tipo 'string'
if columnas_string:
    print("Estadísticas para columnas de tipo 'string':")
    df_merge.select(columnas_string).summary().show()
else:
    print("No hay columnas de tipo 'string' en el DataFrame.")

Estadísticas para columnas numéricas:
+-------+----------------------+------------------+-------------------------------+------------------------+----------------------------+-------------------------+------------------+
|summary|msf_annualizedquota__c|    msf_LTVCont__c|msf_ScoringRFVRecurringDonor__c|msf_AverageDonorValue__c|msf_changeannualizedquota__c|msf_newannualizedquota__c|               age|
+-------+----------------------+------------------+-------------------------------+------------------------+----------------------------+-------------------------+------------------+
|  count|               2003017|           1942916|                        2003017|                  747082|                     2003017|                  2003017|           1612506|
|   mean|     184.4650120543163|1931.3332818660201|              3.036253811125628|        79.5151679333741|          12069.958700100902|        8096.571041059566|57.796179983206265|
| stddev|    1455.8497391508722| 4362.323871908

In [27]:
df_merge.printSchema()

root
 |-- rec_id: string (nullable = true)
 |-- msf_annualizedquota__c: double (nullable = true)
 |-- msf_cancelationdate__c: date (nullable = true)
 |-- msf_currentleadsource1__c: string (nullable = true)
 |-- msf_currentquotamodification__c: string (nullable = true)
 |-- msf_leadsource1__c: string (nullable = true)
 |-- msf_memberid__c: string (nullable = true)
 |-- npe03__contact__c: string (nullable = true)
 |-- npe03__date_established__c: date (nullable = true)
 |-- npe03__installment_period__c: string (nullable = true)
 |-- npe03__recurring_donation_campaign__c: string (nullable = true)
 |-- msf_birthyear__c: string (nullable = true)
 |-- Gender__c: string (nullable = true)
 |-- msf_LTVCont__c: double (nullable = true)
 |-- msf_PressureComplaint__c: boolean (nullable = true)
 |-- msf_ScoringRFVRecurringDonor__c: double (nullable = true)
 |-- msf_AverageDonorValue__c: double (nullable = true)
 |-- con_id: string (nullable = true)
 |-- msf_EntryCampaign__c: string (nullable = true)

In [60]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark.sql.functions import to_date

dataframes_to_union = []

# Definir una lista de fechas límite que te interesan, el 2022 sólo lo usamos para ver si incrementa cuota en ese año pero no para analizarlo dado que el 2023 no está comepleto
fechas_limite = ['2015-12-31', '2016-12-31', '2017-12-31', '2018-12-31', '2019-12-31', '2020-12-31', '2021-12-31', '2022-12-31']

# Iterar para cada año y crear las tablas respectivas
for i in range(len(fechas_limite) -1):
    # Filtrar registros activos para el año correspondiente
    fecha_inicial = fechas_limite[i].split('-')[0] + '-01-01'
    fecha_inicial_col = lit(int(fecha_inicial.split('-')[0]))  # Obtener el año como un valor numérico
    fecha_baja_1Y = fechas_limite[i + 1]

    filas_activos_ano = df_merge.filter(
        (col("npe03__date_established__c") <= fechas_limite[i]) &
        (col("npe03__date_established__c") >= fecha_inicial) &
        ((col("msf_CancelationDate__c") > fechas_limite[i]) | col("msf_CancelationDate__c").isNull())
    )

    # Crear la columna 'flag_aumento' basada en 'msf_changetype__c'

    df_resultado = filas_activos_ano.withColumn(
        'flag_aumento',
        when(
            (filas_activos_ano['msf_changetype__c'] == 'Increase') &
            ((filas_activos_ano["msf_changedate__c"] <= fecha_baja_1Y) & (filas_activos_ano["msf_changedate__c"] > fechas_limite[i])),
            lit(1)
        ).otherwise(lit(0))
    )
    # Crea el campo year_activo para saber en que año estmos mirando
    df_resultado = df_resultado.withColumn('year_activo', fecha_inicial_col)

    # Crear el nombre de la tabla para el año correspondiente
    nombre_tabla = 'df_' + fecha_inicial.split('-')[0]
   
    # Crear la tabla correspondiente
    globals()[nombre_tabla] = df_resultado

    # Añadir el DataFrame resultante a la lista
    dataframes_to_union.append(globals()[nombre_tabla])
    print(nombre_tabla)

# Unir todos los DataFrames en la lista
df_union = dataframes_to_union[0]  # Tomar el primer DataFrame

for df in dataframes_to_union[1:]:
    df_union = df_union.unionAll(df)

df_2015
df_2016
df_2017
df_2018
df_2019
df_2020
df_2021


In [61]:
# Veo con cuantos registros me he quedado:
original_count = df_2015.count()
no_duplicates_count = df_2015.dropDuplicates().count()
print("Cantidad de filas originales:", original_count)
print("Cantidad de filas después de eliminar duplicados:", no_duplicates_count)
print("Filas duplicadas encontradas:", original_count - no_duplicates_count)

Cantidad de filas originales: 127353
Cantidad de filas después de eliminar duplicados: 127353
Filas duplicadas encontradas: 0


In [62]:
# Veo con cuantos registros me he quedado:
original_count = df_2016.count()
no_duplicates_count = df_2016.dropDuplicates().count()
print("Cantidad de filas originales:", original_count)
print("Cantidad de filas después de eliminar duplicados:", no_duplicates_count)
print("Filas duplicadas encontradas:", original_count - no_duplicates_count)

Cantidad de filas originales: 117742
Cantidad de filas después de eliminar duplicados: 117742
Filas duplicadas encontradas: 0


In [63]:
# Veo con cuantos registros me he quedado por año activo:
original_count = df_2017.count()
no_duplicates_count = df_2017.dropDuplicates().count()
print("Cantidad de filas originales:", original_count)
print("Cantidad de filas después de eliminar duplicados:", no_duplicates_count)
print("Filas duplicadas encontradas:", original_count - no_duplicates_count)

Cantidad de filas originales: 108021
Cantidad de filas después de eliminar duplicados: 108021
Filas duplicadas encontradas: 0


In [64]:
# Veo con cuantos registros me he quedado:
original_count = df_2018.count()
no_duplicates_count = df_2018.dropDuplicates().count()
print("Cantidad de filas originales:", original_count)
print("Cantidad de filas después de eliminar duplicados:", no_duplicates_count)
print("Filas duplicadas encontradas:", original_count - no_duplicates_count)

Cantidad de filas originales: 77277
Cantidad de filas después de eliminar duplicados: 77277
Filas duplicadas encontradas: 0


In [65]:
# Veo con cuantos registros me he quedado:
original_count = df_2019.count()
no_duplicates_count = df_2019.dropDuplicates().count()
print("Cantidad de filas originales:", original_count)
print("Cantidad de filas después de eliminar duplicados:", no_duplicates_count)
print("Filas duplicadas encontradas:", original_count - no_duplicates_count)

Cantidad de filas originales: 65402
Cantidad de filas después de eliminar duplicados: 65402
Filas duplicadas encontradas: 0


In [66]:
# Veo con cuantos registros me he quedado:
original_count = df_2020.count()
no_duplicates_count = df_2020.dropDuplicates().count()
print("Cantidad de filas originales:", original_count)
print("Cantidad de filas después de eliminar duplicados:", no_duplicates_count)
print("Filas duplicadas encontradas:", original_count - no_duplicates_count)

Cantidad de filas originales: 53060
Cantidad de filas después de eliminar duplicados: 53060
Filas duplicadas encontradas: 0


In [67]:
# Veo con cuantos registros me he quedado:
original_count = df_2021.count()
no_duplicates_count = df_2021.dropDuplicates().count()
print("Cantidad de filas originales:", original_count)
print("Cantidad de filas después de eliminar duplicados:", no_duplicates_count)
print("Filas duplicadas encontradas:", original_count - no_duplicates_count)

Cantidad de filas originales: 48998
Cantidad de filas después de eliminar duplicados: 48998
Filas duplicadas encontradas: 0


In [68]:
# Veo con cuantos registros me he quedado (suma total) OK:
original_count = df_union.count()
no_duplicates_count = df_union.dropDuplicates().count()
print("Cantidad de filas originales:", original_count)
print("Cantidad de filas después de eliminar duplicados:", no_duplicates_count)
print("Filas duplicadas encontradas:", original_count - no_duplicates_count)

Cantidad de filas originales: 597853
Cantidad de filas después de eliminar duplicados: 597853
Filas duplicadas encontradas: 0


### Guardamos la tabla df_union en S3 en formato parquet y en formto csv para llevarlo a local

In [ ]:
(
  df_union.write
        .format("parquet")
        .mode("overwrite")
        .save(f"s3://{BUCKET_NAME}/output/df_union")
)

In [ ]:
(
  df_union.coalesce(1).write
    .format("csv")
    .option("compression", "gzip")
    .option("header", True)
    .mode("overwrite")
    .save(f"s3://{BUCKET_NAME}/output/df_union_csv")
)

In [2]:
df_union = (
  spark.read
    .option("header", "true")
    .option("quote", "\"")
    .option("escape" , "\"")   
    .format("parquet")
    .load(f"s3://{BUCKET_NAME}/output/df_union")
)

In [3]:
# Filtrar aquellos con flag_aumento igual a 1
registros_flag_1 = df_union.filter(df_union['rec_id'] == 'a093Y00001ReiWeQAJ')

# Mostrar los registros con flag_aumento igual a 1
registros_flag_1.show()

+------------------+----------------------+----------------------+-------------------------+-------------------------------+------------------+---------------+------------------+--------------------------+----------------------------+-------------------------------------+----------------+---------+--------------+------------------------+-------------------------------+------------------------+------------------+--------------------+---------------------------------------+--------------------------------+------------------+---------+------------------------+------------------------+----------------------------+-----------------+------------------+-------------------------+-------------------------+------------------+-----------------+------------+-----------+
|            rec_id|msf_annualizedquota__c|msf_cancelationdate__c|msf_currentleadsource1__c|msf_currentquotamodification__c|msf_leadsource1__c|msf_memberid__c| npe03__contact__c|npe03__date_established__c|npe03__installment_period_

In [4]:
# Filtrar aquellos con flag_aumento igual a 1
registros_flag_1 = df_union.filter((df_union['year_activo'] == 2017) & ( df_union['msf_changetype__c'] =='Decrease'))

# Mostrar los registros con flag_aumento igual a 1
registros_flag_1.show(5)

+------------------+----------------------+----------------------+-------------------------+-------------------------------+------------------+---------------+------------------+--------------------------+----------------------------+-------------------------------------+----------------+---------+--------------+------------------------+-------------------------------+------------------------+------------------+--------------------+---------------------------------------+--------------------------------+------------------+---------+------------------------+------------------------+----------------------------+-----------------+------------------+-------------------------+-------------------------+------------------+-----------------+------------+-----------+
|            rec_id|msf_annualizedquota__c|msf_cancelationdate__c|msf_currentleadsource1__c|msf_currentquotamodification__c|msf_leadsource1__c|msf_memberid__c| npe03__contact__c|npe03__date_established__c|npe03__installment_period_

In [28]:
# Filtrar aquellos con flag_aumento igual a 1
registros_flag_1 = df_resultado_final.filter((df_resultado_final['flag_aumento'] == 1) & (df_resultado_final["msf_changedate__c"] >= '2017-01-01') & (df_resultado_final["npe03__date_established__c"] == '2016-12-31'))

# Mostrar los registros con flag_aumento igual a 1
registros_flag_1.show(5)


+------------------+----------------------+----------------------+-------------------------+-------------------------------+------------------+---------------+------------------+--------------------------+----------------------------+-------------------------------------+----------------+---------+--------------+------------------------+-------------------------------+------------------------+------------------+--------------------+---------------------------------------+--------------------------------+------------------+---------+------------------------+------------------------+----------------------------+-----------------+------------------+-------------------------+-------------------------+------------------+-----------------+------------+
|            rec_id|msf_annualizedquota__c|msf_cancelationdate__c|msf_currentleadsource1__c|msf_currentquotamodification__c|msf_leadsource1__c|msf_memberid__c| npe03__contact__c|npe03__date_established__c|npe03__installment_period__c|npe03__re

In [8]:
# Veo con cuantos registros me he quedado:
original_count = df_resultado_final.count()
no_duplicates_count = df_resultado_final.dropDuplicates().count()
print("Cantidad de filas originales:", original_count)
print("Cantidad de filas después de eliminar duplicados:", no_duplicates_count)
print("Filas duplicadas encontradas:", original_count - no_duplicates_count)

Cantidad de filas originales: 3152277
Cantidad de filas después de eliminar duplicados: 1382092
Filas duplicadas encontradas: 1770185
